In [1]:
import pandas as pd

### Simulation Input Data

| Variable           | Description                                                                                                                         |
|--------------------|-------------------------------------------------------------------------------------------------------------------------------------|
| CASENO             | A case identifier used to track different instances or runs within the simulation. (Distinguishes between different scenarios/simulation conditions) |
| PROJECTNO          | The project number, identifying a specific project within the simulation. Used for associating activities, durations, and resources with the correct project |
| ACTIVITYCODE       | Specifies a particular activity within a project |
| LOCATIONCODE       | Indicates the location |
| LOCATIONTYPE       | Describes the type of location|
| WORKTYPE           | The category of work being performed  |
| MAINPROCESSCODE    | Breaks down activities into main processes within projects |
| SUBPROCESSCODE     | Further categorizes activities into sub-processes |
| SHOPCODE           | Identifies specific workshops or areas within projects where work is executed, |
| ACTUALDURATION     | The actual time taken to complete activities |
| VOL1, VOL2, VOL3   | Represents different volumes of resources used or produced by activities (e.g., materials, waste, output), measured in various units |
| OFFSET             | Refers to a scheduling or positioning offset, related to timing or spatial layout within the project's timeline or physical space   |
| DESCUPDOWN         | Indicates vertical positioning or changes in setup |
| LINEPOSITION       | Specifies a position within a sequence or assembly line |
| ACTIVITYDESC       | Provides a descriptive label or name for activities |
| WORKCODE           | Further categorizes the type of work within its broader classification |
| STAGECODE          | Indicates the stage or phase of the project that activities belong to |
| NOTEPOSITION       | Describes specific notes or annotations |
| REFKEYEVENT        | References key events or milestones linked to activities, for critical path analysis and milestone tracking in project |


In [6]:
file_path = 'data/MCM_ACTIVITY.xls'
data = pd.read_excel(file_path)
filtered_data = data.dropna(axis=1, how='all')
filtered_data.head()

,CASENO,PROJECTNO,ACTIVITYCODE,LOCATIONCODE,LOCATIONTYPE,WORKTYPE,MAINPROCESSCODE,SUBPROCESSCODE,SHOPCODE,ACTUALDURATION,...,VOL2,VOL3,OFFSET,DESCUPDOWN,LINEPOSITION,ACTIVITYDESC,WORKCODE,STAGECODE,NOTEPOSITION,REFKEYEVENT
0,DHSCMID,U611,A11C HA041,A11C,B,H,A,04,AS2,0.0,...,0.0,0.0,0,U,0.0,대조 / IN SHOP,5.0,1,NaN,NaN
1,DHSCMID,U611,A11C HF031,A11C,B,H,F,03,AS2,0.0,...,0.0,0.0,0,U,0.0,가공 / IN SHOP,2.0,1,NaN,NaN
2,DHSCMID,U611,A11C PB011,A11C,B,P,B,01,AS2,0.0,...,0.0,0.0,0,U,0.0,블럭도장 / IN SHOP,9.0,1,NaN,NaN
3,DHSCMID,U611,A11C HF010,A11C,B,H,F,01,AS2,0.0,...,0.0,0.0,0,U,0.0,DWG / 공통,NaN,0,NaN,NaN
4,DHSCMID,U611,A11C HF020,A11C,B,H,F,02,AS2,0.0,...,0.0,0.0,0,U,0.0,강재입고 / 공통,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6699,DHSCMID,U614,T18S HL034,T18S,B,H,L,03,AS2,0.0,...,0.0,0.0,0,U,0.0,2차 PE Lifting / 2차 PE,6.0,4,NaN,NaN
6700,DHSCMID,U614,T18S HF031,T18S,B,H,F,03,AS2,0.0,...,0.0,0.0,0,U,0.0,가공 / IN SHOP,2.0,1,NaN,NaN
6701,DHSCMID,U614,T18S HA041,T18S,B,H,A,04,AS2,0.0,...,0.0,0.0,0,U,0.0,대조 / IN SHOP,5.0,1,NaN,NaN
6702,DHSCMID,U614,T18S FB021,T18S,B,F,B,02,AS2,0.0,...,0.0,0.0,0,U,0.0,선행의장 / IN SHOP,11.0,1,NaN,NaN


In [36]:
activity_desc = filtered_data["ACTIVITYDESC"].value_counts()
print(f"Activty Desciprtions: {len(activity_desc)}")
activity_code = filtered_data["ACTIVITYCODE"].value_counts()
print(f"Activity Codes: {len(activity_code)}")
activity_desc.head(10)

# Pre-assembly / IN SHOP 508
# Processing / IN SHOP 504
# DWG / Common 504
# Steel reentry / Common 504
# Comparison / IN SHOP 492
# Sub-assembly (for comparison) / IN SHOP 492
# 1st PE Lifting / 1st PE 376
# Medium assembly / IN SHOP 376
# Medium assembly / IN SHOP 366
# Sub-assembly (for medium assembly) / IN SHOP 360

Activty Desciprtions: 248
Activity Codes: 1678


선행의장 / IN SHOP           508
가공 / IN SHOP             504
DWG / 공통                 504
강재입고 / 공통                504
대조 / IN SHOP             492
소조(대조용) / IN SHOP        492
1차 PE Lifting / 1차 PE    376
중조의장 / IN SHOP           376
중조 / IN SHOP             366
소조(중조용) / IN SHOP        360
Name: ACTIVITYDESC, dtype: int64

Overview of Key Components
    DataframePart Class: Represents an individual data block that moves through the system. It contains attributes like time, id, source (src), destination (dst), and activity data which includes process specific information such as task durations.
    DataframeSource Class: Acts as a source that generates blocks at a given inter-arrival time (IAT). It simulates the creation of blocks until a specified finish time. Each block is then handed off to the first process in the workflow based on the activity data.
    Process Class: Represents a processing step in the workflow. Each process can handle a specific number of blocks simultaneously (subprocess_num). It processes each block for a predetermined time, and then either passes it on to the next process or finishes handling it.
    Sink Class: This is where blocks end up after completing all processes. The Sink class can record arrival times, wait times, and can be set to debug mode to provide detailed logs of the blocks being processed.

Simulation Flow:
    Initialization: Each block is initialized with data and settings (like arrival times and processing times).
    Block Generation: Blocks are generated at the source with set inter-arrival times and passed to the initial process.
    Processing: Each block is processed according to the specific processing times in each process. If a process is at capacity, the block waits.
    Completion: After a block has gone through all necessary processes, it reaches the Sink, where final data can be recorded.

In [44]:
import simpy
import random
from SimComponents_for_supply_chain import DataframePart, Source, Sink, Process


# Dummy DataFrame for illustration
df = pd.DataFrame({
    'ct1': [2, 3, 5, 7]  # Example processing times for the parts
})

class CustomSource(Source):
    def __init__(self, env, id, adist, df, initial_delay=0, finish=float("inf"), flow_id=0):
        super().__init__(env, id, adist, initial_delay, finish, flow_id)
        self.df = df

    def run(self):
        yield self.env.timeout(self.initial_delay)
        while self.env.now < self.finish:
            yield self.env.timeout(self.adist())
            self.parts_sent += 1
            idx = (self.parts_sent - 1) % len(self.df)
            p = DataframePart(self.env.now, self.df.iloc[idx], self.parts_sent, src=self.id, flow_id=self.flow_id)
            self.out.put(p)

def simulation(env):
    source = CustomSource(env, id='Source1', adist=lambda: random.randint(1, 5), df=df)
    process = Process(env, process_kind='proc1', name='Machining', rate=lambda: 2, subprocess_num=1, qlimit=10)
    sink = Sink(env, name='FinishedGoods', debug=True)
    
    source.out = process
    process.outs['Sink'] = sink
    
    env.run(until=100)

# Set up the simulation environment
env = simpy.Environment()
simulation(env)
print("Simulation completed.")


TypeError: unsupported type for timedelta minutes component: numpy.int64